In [1]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms

In [2]:
import os
import pandas as pd
os.chdir('..')

X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

# Example model: A simple feedforward neural network for regression
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Define the model
input_size = 10  # Adjust this based on your actual input features size
hidden_size = 50
output_size = 1  # For regression tasks
model = SimpleNN(input_size, hidden_size, output_size)

# Define the loss function (Mean Absolute Error)
criterion = nn.L1Loss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Print model summary (optional)
print(model)

SimpleNN(
  (fc1): Linear(in_features=10, out_features=50, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=50, out_features=1, bias=True)
)


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Custom dataset
class CustomDataset(Dataset):
    def __init__(self, features, labels, transform=None):
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]

        if self.transform:
            feature = self.transform(feature)

        return feature, label



# Create custom datasets
train_dataset = CustomDataset(X_train, y_train, transform=None)
test_dataset = CustomDataset(X_test, y_test, transform=None)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

# Move model to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
print('Starting training....')
for epoch in range(10):
    running_loss = 0.0
    for i, (data, target) in enumerate(train_loader):
        data, target = data.float().to(device), target.float().to(device).view(-1, 1)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 1 == 0:  # Print every 100 batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 1))
            running_loss = 0.0

# Save final model
torch.save(model.state_dict(), 'final_model.pth')


Starting training....
